In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import Sequence
from osgeo import gdal
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
import pennylane as qml
from keras.models import load_model
from keras.utils import get_custom_objects
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, Layer

2024-04-02 13:03:35.993970: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 13:03:36.029503: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 13:03:36.029535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 13:03:36.030391: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-02 13:03:36.036172: I tensorflow/core/platform/cpu_feature_guar

In [2]:
class TiffImageDataGenerator(Sequence):
    def __init__(self, image_files, labels, batch_size):
        self.image_files = image_files
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.image_files) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.image_files[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]

        return np.array([
            self.preprocess_image(file_name) for file_name in batch_x]), np.array(batch_y)

    def preprocess_image(self, file):
        dataset = gdal.Open(file)
        channels = [dataset.GetRasterBand(i + 1).ReadAsArray() for i in range(dataset.RasterCount)]
        image = np.stack(channels, axis=-1)
        image = cv2.resize(image, (64, 64))
        image = image / 255.0
        return image

dataset_path = "/home/admin1/Selvin/BE/EuroSAT_MS/"
subdirs = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
image_files = []
labels = []
label = 0
for subdir in subdirs:
    image_files_subdir = [os.path.join(dataset_path, subdir, f) for f in os.listdir(os.path.join(dataset_path, subdir)) if f.endswith(".tif")]
    image_files.extend(image_files_subdir)
    labels.extend([label]*len(image_files_subdir))
    label += 1

# Define a dictionary to map the old labels to the new ones
# Define a dictionary to map the old labels to the new ones
label_mapping = {
    'SeaLake': 'Water_Bodies',
    'River': 'Water_Bodies',
    'HerbaceousVegetation': 'Vegetation',
    'PermanentCrop': 'Vegetation',
    'AnnualCrop': 'Vegetation',
    'Pasture': 'Vegetation',
    'Forest': 'Vegetation',
    'Industrial': 'Urban',
    'Highway': 'Urban',
    'Residential': 'Urban'
}

# Update the labels
new_labels = []
image_files = []  # Initialize the image_files list
for subdir in subdirs:
    image_files_subdir = [os.path.join(dataset_path, subdir, f) for f in os.listdir(os.path.join(dataset_path, subdir)) if f.endswith(".tif")]
    image_files.extend(image_files_subdir)
    new_labels.extend([label_mapping[subdir]]*len(image_files_subdir))

# Now, you can use `new_labels` in place of `labels` for your train-test split and data generator
# Define a dictionary to map the new string labels to numerical labels
str_to_num_mapping = {
    'Water_Bodies': 0,
    'Vegetation': 1,
    'Urban': 2
}

# Convert the string labels to numerical labels
num_labels = [str_to_num_mapping[label] for label in new_labels]

# Now, you can use `num_labels` in place of `new_labels` for your train-test split and data generator
X_train, X_test, y_train, y_test = train_test_split(image_files, num_labels, test_size=0.2, random_state=42)

# Create a TiffImageDataGenerator instance for training and testing data
train_gen = TiffImageDataGenerator(X_train, y_train, batch_size=32)
test_gen = TiffImageDataGenerator(X_test, y_test, batch_size=32)


In [5]:
class Quantum_No(Layer):
    def __init__(self, **kwargs):
        super(Quantum_No, self).__init__(**kwargs)
        self.dev = qml.device("default.qubit", wires=4)

    def build(self, input_shape):
        super(Quantum_No, self).build(input_shape)

    def call(self, inputs):
        # Convert inputs to tensor
        inputs = tf.convert_to_tensor(inputs)
        # Compute quantum circuit results
        output = tf.vectorized_map(self.quantum_func, inputs)
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 16)

    def quantum_func(self, inputs):
        @qml.qnode(self.dev, interface='tf')
        def quantum_circuit(params):
            qml.Hadamard(wires=0)
            qml.Hadamard(wires=1)
            qml.Hadamard(wires=2)
            qml.Hadamard(wires=3)

            # Use the values in the batch item for the initial rotation (theta1)
            for i in range(4):
                qml.RY(params[i], wires=i)

            return qml.probs(wires=[0, 1, 2, 3])

        # Compute quantum circuit results
        probs = quantum_circuit(inputs)
        return probs

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 13)))  # Assuming your images have 3 channels
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(4, activation='relu'))
model.add(BatchNormalization())
model.add(Quantum_No())
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))  # Assuming you have 10 classes

In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
epochs = 30

# Define the ModelCheckpoint callback to save the best model based on validation accuracy
checkpoint_callback = ModelCheckpoint(
    "best_model.h5",
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Define the EarlyStopping callback to stop training if there's no improvement in validation accuracy
early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',
    patience=10,  # Number of epochs with no improvement after which training will be stopped
    verbose=1,
    restore_best_weights=True
)

print_loss_accuracy_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: print(
        f" Epoch {epoch + 1}/{epochs} - "
        f" Training Loss: {logs['loss']:.4f}, "
        f" Validation Loss: {logs['val_loss']:.4f}, "
        f" Training Accuracy: {logs['accuracy'] * 100:.2f}%, "
        f" Validation Accuracy: {logs['val_accuracy'] * 100:.2f}%"
    )
)

# Train the model using the generators and include the callbacks
history = model.fit(train_gen, validation_data=test_gen, epochs=epochs, callbacks=[print_loss_accuracy_callback, checkpoint_callback, early_stopping_callback])

# model.fit(train_gen, validation_data=test_gen, epochs=epochs, callbacks=[print_loss_accuracy_callback, checkpoint_callback])

# Load the best model based on validation accuracy
best_model = load_model("best_model.h5", custom_objects={'Quantum_No': Quantum_No})

# Save the best model with a different name, for example, "PQC_Circuit"
best_model.save("Three_Categories_Quantum_No.h5")

Epoch 1/30


/home/admin1/miniconda3/lib/python3.11/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


2024-04-02 13:04:31.374132: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-04-02 13:04:31.500702: E external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-04-02 13:04:31.502024: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:225] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2024-04-02 13:04:31.502046: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:228] Used ptxas at ptxas
2024-04-02 13:04:31.502093: W external/local_xla/xla/stream_executor/gpu/redzone_allocator.cc:322] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-04-02 13:04

  1/675 [..............................] - ETA: 1:15:58 - loss: 1.1772 - accuracy: 0.3438

2024-04-02 13:04:35.433585: E external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



674/675 [============================>.] - ETA: 0s - loss: 0.4076 - accuracy: 0.8407 Epoch 1/30 -  Training Loss: 0.4071,  Validation Loss: 0.2344,  Training Accuracy: 84.09%,  Validation Accuracy: 93.39%

Epoch 1: val_accuracy improved from -inf to 0.93389, saving model to best_model.h5
675/675 [==============================] - 40s 50ms/step - loss: 0.4071 - accuracy: 0.8409 - val_loss: 0.2344 - val_accuracy: 0.9339
Epoch 2/30


/home/admin1/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


675/675 [==============================] - ETA: 0s - loss: 0.1807 - accuracy: 0.9397 Epoch 2/30 -  Training Loss: 0.1807,  Validation Loss: 1.1481,  Training Accuracy: 93.97%,  Validation Accuracy: 69.15%

Epoch 2: val_accuracy did not improve from 0.93389
675/675 [==============================] - 27s 39ms/step - loss: 0.1807 - accuracy: 0.9397 - val_loss: 1.1481 - val_accuracy: 0.6915
Epoch 3/30
675/675 [==============================] - ETA: 0s - loss: 0.1472 - accuracy: 0.9527 Epoch 3/30 -  Training Loss: 0.1472,  Validation Loss: 0.1538,  Training Accuracy: 95.27%,  Validation Accuracy: 94.41%

Epoch 3: val_accuracy improved from 0.93389 to 0.94407, saving model to best_model.h5
675/675 [==============================] - 25s 36ms/step - loss: 0.1472 - accuracy: 0.9527 - val_loss: 0.1538 - val_accuracy: 0.9441
Epoch 4/30
674/675 [============================>.] - ETA: 0s - loss: 0.1243 - accuracy: 0.9628 Epoch 4/30 -  Training Loss: 0.1242,  Validation Loss: 0.1673,  Training Accur

NameError: name 'QuantumNo' is not defined

In [9]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_gen)

# Convert predictions to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_labels))

169/169 [==============================] - 6s 34ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1100
           1       0.98      0.99      0.99      2682
           2       0.98      0.97      0.98      1618

    accuracy                           0.98      5400
   macro avg       0.98      0.98      0.98      5400
weighted avg       0.98      0.98      0.98      5400

